In [1]:
from utils_mar import *

In [5]:
from IPython import display
from utilsNew import *
from utils import *
import random
import numpy as np
import torch
import pickle
import timeit
import time
import argparse
import pprint
from pathlib import Path
from confs import fn, fn2, fn22, LogFn
#from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
import numpy.random as npr

In [3]:
import matplotlib.pyplot as plt
import pickle

In [4]:
cudaid = 2
loglv = 2
torch.cuda.set_device(cudaid)
#------------------------------------------------------------------------------------
# fix the random seed for several packages
torch.manual_seed(0) # cpu
torch.cuda.manual_seed(0) #gpu
np.random.seed(0) #numpy
random.seed(0) #random and transforms
torch.backends.cudnn.deterministic=True # cudnn

In [5]:
cuda = torch.cuda.is_available()
if cuda:
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)

In [6]:
conDenfs = [fn, fn2, fn22]
f = fn
f2 = fn2

In [7]:
n = m = 400
p = 50
prob = 0.2

sigmaY = 1.0

# generate the parameters
beta0 = torch.cat((torch.tensor([1.0, 0, 2, 0, -3, -4, 5]), torch.zeros(p-7))) 
bTheta0 = genbTheta(n, m, sigVs=np.array([10, 9, 8, 7, 6])*10) 
initbetapref = 1 + (torch.rand(p)-1/2)/40  #[0.875, 1.125]
initthetapref = 1 + (torch.rand(n, m)-1/2)/4
betainit = beta0 * initbetapref
bThetainit = bTheta0 * initthetapref
#betainit = torch.cat((torch.tensor([0.8, 0, 1.5, 0, -2.5, -4.3, 4.8]), torch.zeros(p-7))) * 10
#bThetainit = genbTheta(n, m, sigVs=np.array([15, 10, 7, 5, 5])/2) 

#X = genXUnif(n, m, p) 
X = genXBin(n, m, p, prob=prob) 
Y = genYnorm(X, bTheta0, beta0, sigmaY)
# around 95% missing rate
inps = {}
inps[50] = - 8.0
inps[100] =  -5.93
inps[200] = -5.30
inps[400] = -5.2
inps[800] = -5.15
inps[1600] = -5.1
R = genR(Y, "linear", inp=inps[n], slop=1)
R.to_dense().mean()

tensor(0.0502)

In [8]:
v1 = lossLpb(bTheta0, beta0, conDenfs, X, Y, R, fct=10, N=10000)

In [10]:
v1 = marLossLpT(bTheta0, beta0, conDenfs, X, Y, R)
v2 = lossLpT(bTheta0, beta0, conDenfs, X, Y, R)

In [14]:
torch.norm(v1)

tensor(0.0007)

In [15]:
torch.norm(v1  - v2)

tensor(0.0004)

In [106]:
etaT = 0.1
CT = 20

In [107]:
bThetaOld = bTheta0
betaNew = beta0
LamT = LamTfn(CT, n, m, p)

In [108]:
LpTvOld = lossLpT(bThetaOld, betaNew, conDenfs, X, Y, R)
svdres = torch.svd(bThetaOld-LpTvOld*etaT)
U, S, V =  svdres.U, svdres.S, svdres.V
softS = (S-LamT*etaT).clamp_min(0)
bThetaNew1 = U.matmul(torch.diag(softS)).matmul(V.t())

In [109]:
bThetaOld = bTheta0
betaNew = beta0
LamT = LamTfn(CT, n, m, p)

In [110]:
LpTvOld = marLossLpT(bThetaOld, betaNew, conDenfs, X, Y, R)
svdres = torch.svd(bThetaOld-LpTvOld*etaT)
U, S, V =  svdres.U, svdres.S, svdres.V
softS = (S-LamT*etaT).clamp_min(0)
bThetaNew2 = U.matmul(torch.diag(softS)).matmul(V.t())

In [111]:
torch.norm(bThetaNew1 - bThetaNew2)

tensor(0.0004)